In [1]:
import os

os.chdir("..")
print(f"Changed working directory to: {os.getcwd()}")

Changed working directory to: /home/jovyan/work/FlareSense


In [2]:
import torch
import src.utils.data as data

from torchvision import transforms
from pprint import pprint

### Beispiel eines Data Loaders und deren Daten

In [3]:
data_folder_path = "data/raw/burst_images/"

data_module = data.ECallistoDataModule(
    data_folder=data_folder_path,
    transform=transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((193, 240), antialias=True),
            transforms.ToTensor(),
        ]
    ),
    batch_size=32,
    num_workers=0,
    val_ratio=0.15,
    test_ratio=0.15,
    split_by_date=True,
    filter_instruments=["australia_assa_02"],
)
data_module.setup()

Not enough validation bursts (97), minimum 119 needed
Reshuffling...

Not enough validation bursts (46), minimum 119 needed
Not enough test bursts (99), minimum 119 needed
Reshuffling...

Not enough validation bursts (86), minimum 119 needed
Reshuffling...

Too many test bursts (396), maximum 266 allowed
Reshuffling...

Dataset split successfully
Train:		863 bursts
Validation:	148 bursts
Test:		172 bursts


### Overlap überprüfen

In [4]:
# check overlaps train and val
files_train = data_module.train_dataset.observations["file_path"].to_list()
files_val = data_module.val_dataset.observations["file_path"].to_list()
print(f"Intersection in train and val: {len(set(files_train).intersection(set(files_val)))}")

# check overlaps train and test
files_test = data_module.test_dataset.observations["file_path"].to_list()
print(f"Intersection in train and test: {len(set(files_train).intersection(set(files_test)))}")

# check overlaps val and test
print(f"Intersection in val and test: {len(set(files_val).intersection(set(files_test)))}")

Intersection in train and val: 0
Intersection in train and test: 0
Intersection in val and test: 0


### Überprüfung der Batches

In [5]:
train_loader = data_module.train_dataloader()
image, info = next(iter(train_loader))

print(f"Image shape: {image.shape}")
pprint({key: value[0] for key, value in info.items()})

Image shape: torch.Size([32, 1, 193, 240])
{'end': '2021-03-22 04:38:00',
 'file_path': 'data/raw/burst_images/no_burst/2021-03-22 04-37-00_2021-03-22 '
              '04-38-00_australia_assa_02_None_no_burst.png',
 'instrument': 'australia_assa_02',
 'label': 'no_burst',
 'start': '2021-03-22 04:37:00'}
